# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
%load_ext dotenv
%dotenv 



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [10]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
import os
from glob import glob

price_dir = os.getenv("PRICE_DATA")
price_glob = glob(os.path.join(price_dir,"**/*.parquet"), recursive=True)

In [12]:
dd_price = dd.read_parquet(price_glob).set_index("Ticker")

In [13]:
dd_price

,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=13078,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [14]:
dd_price.info()

<class 'dask_expr.DataFrame'>
Columns: 8 entries, Date to Year
dtypes: datetime64[ns, UTC](1), float64(6), int32(1)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [15]:
dd_feat =(dd_price
          .groupby('Ticker', group_keys=False)
          .apply(
                lambda x: x.assign(
                    Close_lag_1 = x['Close'].shift(1),
                    Adj_Close_lag_1 = x['Adj Close'].shift(1),
                    returns = x['Close']/x['Close'].shift(1)-1,
                    hi_lo_range = x['High'] - x['Low'],
                    )              
    )
)

dd_feat

C:\Users\jcp_2\AppData\Local\Temp\ipykernel_26348\2358109198.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat =(dd_price


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=13078,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [16]:
df_feat = dd_feat.compute()



In [24]:
df_feat = df_feat.sort_values(["Ticker", "Date"])
df_feat["returns_ma_10"] = (
    df_feat
    .groupby("Ticker")["returns"]
    .transform(lambda x: x.rolling(10).mean())
)

In [25]:
df_feat.head(15)

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
Ticker,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.382843,51.502148,56.464592,48.193848,56.330471,4674353.0,2000,40.908440,37.318741,0.258961,8.270744,NaN
A,2000-01-04 00:00:00+00:00,40.068882,47.567955,49.266811,46.316166,48.730328,4765083.0,2000,51.502148,43.382843,-0.076389,2.950645,NaN
A,2000-01-05 00:00:00+00:00,37.583405,44.617310,47.567955,43.141991,47.389126,5758642.0,2000,47.567955,40.068882,-0.062030,4.425964,NaN
A,2000-01-06 00:00:00+00:00,36.152363,42.918453,44.349072,41.577251,44.080830,2534434.0,2000,44.617310,37.583405,-0.038076,2.771820,NaN
A,2000-01-07 00:00:00+00:00,39.165054,46.494991,47.165592,42.203148,42.247852,2819626.0,2000,42.918453,36.152363,0.083333,4.962444,NaN
A,2000-01-10 00:00:00+00:00,41.537560,49.311516,49.803291,48.327969,49.356224,2148446.0,2000,46.494991,39.165054,0.060577,1.475323,NaN
A,2000-01-11 00:00:00+00:00,40.972683,48.640915,49.311516,47.523247,49.311516,1855985.0,2000,49.311516,41.537560,-0.013599,1.788269,NaN
A,2000-01-12 00:00:00+00:00,40.144199,47.657368,48.640915,45.824390,48.640915,1429874.0,2000,48.640915,40.972683,-0.020221,2.816525,NaN
A,2000-01-13 00:00:00+00:00,40.746738,48.372677,49.937412,47.210300,48.909157,1134337.0,2000,47.657368,40.144199,0.015009,2.727112,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1. It is not strictly necessary to convert to Pandas just to compute a moving average. Dask does support rolling windows (with caveats).

2. If your dataset is large or you want to keep everything parallel/out‐of‐core, it’s generally better to stay in Dask, so you don’t over use memory by converting to Pandas.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.